In [1]:
import sys
sys.path.append('../../../')
import warnings
warnings.filterwarnings(action='ignore')
import torch
from torch import nn
from train import val        

from static_definer import *

In [2]:
from models.domain_shift.adversarial.functions import DomainDiscriminator, train
from models.bisenet.build_bisenet import BiSeNet

In [3]:

# # defining a CrossEntropyLoss for the segmentation and a BCEWithLogitsLoss for the domain classification
generator_loss = nn.CrossEntropyLoss(ignore_index=19)
discriminator_loss = nn.BCEWithLogitsLoss()

# defining the models
generator = BiSeNet(num_classes=num_classes,context_path='resnet18',with_interpolation=True)
# defining the Discriminator
discriminator = DomainDiscriminator(num_classes=num_classes,with_grl=False)

# defining the optimizer
generator_optimizer = torch.optim.SGD(generator.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-3)

In [4]:
source_input_size = (720, 1280)
target_input_size = (512, 1024)
# These interpolation are defined to resizing the output of the discriminator
# source_interp = nn.Upsample(size=(source_input_size[1], source_input_size[0]), mode='bilinear')
# target_interp = nn.Upsample(size=(target_input_size[1], target_input_size[0]), mode='bilinear')
discriminator_interpolator = nn.Upsample(size=(target_input_size[0], target_input_size[1]), mode='bilinear')

In [5]:
for epoch in range(1, 10):

    train(iterations=100,
          epoch=epoch,
          generator=generator,
          discriminator=discriminator,
          generator_optimizer=generator_optimizer,
          discriminator_optimizer=discriminator_optimizer,
          generator_loss=generator_loss,
          discriminator_loss=discriminator_loss,
          source_dataloader=train_dataloaderGTA5,
          target_dataloader=train_dataloader,
          device=device,
          discriminator_interpolator=discriminator_interpolator,
          image_inter_size=target_input_size)


    if epoch % 5 == 0:
        print('-'*50)
        val(epoch, generator, val_dataloader, num_classes, device)
        print('-'*50)

torch.save(generator.state_dict(), f'generator_{epoch}.pth')
torch.save(discriminator.state_dict(), f'discriminator_{epoch}.pth')
    

Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 